In [19]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import statsmodels.api as sm

# 1A

In [2]:
ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
ff5_month = ff5_month / 100
mom_month = pdr.get_data_famafrench('F-F_Momentum_Factor', start='1927-01', end='2023-03')[0]
mom_month = mom_month / 100

ff5_month = ff5_month.reset_index()
mom_month = mom_month.reset_index()
ff5mom_month = ff5_month.merge(mom_month, on='Date', how='right')
ff5mom_month = ff5mom_month.set_index('Date')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_62612/3245062299.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_62612/3245062299.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_62612/3245062299.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read y

In [8]:
ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
ff5_daily = ff5_daily / 100
mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]
mom_daily = mom_daily / 100

ff5_daily = ff5_daily.reset_index()
mom_daily = mom_daily.reset_index()
ff5mom_daily = ff5_daily.merge(mom_daily, on='Date', how='right')
ff5mom_daily = ff5mom_daily.set_index('Date')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_62612/152997468.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_62612/152997468.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]


In [9]:
ff5mom_month

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1927-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0036
1927-02,NaN,NaN,NaN,NaN,NaN,NaN,-0.0214
1927-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0361
1927-04,NaN,NaN,NaN,NaN,NaN,NaN,0.0430
1927-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0300
...,...,...,...,...,...,...,...
2022-11,0.0460,-0.0267,0.0138,0.0601,0.0311,0.0029,-0.0201
2022-12,-0.0641,-0.0016,0.0132,0.0009,0.0419,0.0033,0.0452
2023-01,0.0665,0.0443,-0.0405,-0.0262,-0.0453,0.0035,-0.1598


In [10]:
ff5mom_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0056
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,-0.0050
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0117
1926-11-06,NaN,NaN,NaN,NaN,NaN,NaN,-0.0003
1926-11-08,NaN,NaN,NaN,NaN,NaN,NaN,-0.0001
...,...,...,...,...,...,...,...
2023-03-27,0.0027,0.0051,0.0102,-0.0028,0.0035,0.00016,0.0082
2023-03-28,-0.0017,-0.0003,0.0074,0.0008,0.0055,0.00016,0.0073
2023-03-29,0.0139,-0.0034,-0.0050,-0.0090,-0.0054,0.00016,-0.0111


# 1B

In [11]:
# Calculate the rolling variance using a window of 22 trading days
rolling_variance_daily = ff5mom_daily.rolling(window=22).var()

In [15]:
rolling_variance_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-03-27,0.000133,0.000042,0.000124,0.000023,0.000032,4.935065e-11,0.000076
2023-03-28,0.000128,0.000043,0.000118,0.000023,0.000033,3.463203e-11,0.000072
2023-03-29,0.000137,0.000042,0.000118,0.000027,0.000034,1.818182e-11,0.000078


# 1C

In [13]:
monthly_data = rolling_variance_daily.resample('M').last()

In [18]:
monthly_data.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   ',
       'lagged_variance'],
      dtype='object')

In [17]:
for factor in monthly_data.columns:
    # Calculate the lagged variance
    monthly_data['lagged_variance'] = monthly_data[factor].shift()

    # Drop any rows with missing values
    monthly_data = monthly_data.dropna()

    # Create the design matrix for the regression
    X = sm.add_constant(monthly_data['lagged_variance'])

    # Fit the regression model
    model = sm.OLS(monthly_data[factor], X)
    results = model.fit()

    # Print the regression results for each factor
    print(f"Results for {factor}:")
    print(results.summary())
    print()

Results for Mkt-RF:
                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     186.5
Date:                Mon, 15 May 2023   Prob (F-statistic):           6.81e-38
Time:                        15:16:32   Log-Likelihood:                 5094.1
No. Observations:                 716   AIC:                        -1.018e+04
Df Residuals:                     714   BIC:                        -1.018e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.725

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_62612/297911896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_data['lagged_variance'] = monthly_data[factor].shift()


It seems to predict returns in all cases but RF.

In [20]:
# Calculate the long-run average monthly variance for each factor
ci = ff5mom_month.var()

# Calculate the sum of long-run average monthly variances
sum_ci = ci.sum()

# Calculate the weight for each factor
wi = ci / sum_ci

# Shift the weights to align with the factor returns (use t-1 weights)
wi_shifted = wi.shift()

# Calculate the volatility-managed portfolio excess returns
volatility_managed_returns = wi_shifted * ff5mom_month

# Calculate the excess returns on the volatility-managed portfolio
excess_returns = volatility_managed_returns.sum(axis=1)

# Combine excess returns with the risk-free rate
portfolio_returns = pd.concat([excess_returns, ff5mom_month.RF], axis=1)
portfolio_returns.columns = ['portfolio_excess_returns', 'risk_free_rate']

# Calculate the final portfolio excess returns by subtracting the risk-free rate
portfolio_returns['portfolio_excess_returns'] = portfolio_returns['portfolio_excess_returns'] - portfolio_returns['risk_free_rate']

In [21]:
portfolio_returns

,portfolio_excess_returns,risk_free_rate
Date,,
1927-01,NaN,NaN
1927-02,NaN,NaN
1927-03,NaN,NaN
1927-04,NaN,NaN
1927-05,NaN,NaN
...,...,...
2022-11,0.001238,0.0029
2022-12,0.001298,0.0033
2023-01,-0.002502,0.0035


# 1f

In [28]:
portfolio_returns = portfolio_returns.dropna()
ff5mom_month = ff5mom_month.dropna()

In [31]:
capm_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month['Mkt-RF']))
capm_results = capm_model.fit()
capm_alpha = capm_results.params[0]

# FF3 Model
ff3_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month[['Mkt-RF', 'SMB', 'HML']]))
ff3_results = ff3_model.fit()
ff3_alpha = ff3_results.params[0]

# FF5+Mom Model
ff5_mom_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month))
ff5_mom_results = ff5_mom_model.fit()
ff5_mom_alpha = ff5_mom_results.params[0]


# Print the alphas
print("CAPM: ", capm_results.summary())
print("FF3:", ff3_results.summary())
print("FF5+Mom:", ff5_mom_results.summary())

CAPM:                                 OLS Regression Results                               
Dep. Variable:     portfolio_excess_returns   R-squared:                       0.006
Model:                                  OLS   Adj. R-squared:                  0.005
Method:                       Least Squares   F-statistic:                     4.302
Date:                      Mon, 15 May 2023   Prob (F-statistic):             0.0384
Time:                              15:34:54   Log-Likelihood:                 2297.0
No. Observations:                       717   AIC:                            -4590.
Df Residuals:                           715   BIC:                            -4581.
Df Model:                                 1                                         
Covariance Type:                  nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------